In [9]:
"""Libraries"""
import requests
import os
from dotenv import load_dotenv
import time
from datetime import datetime
import csv

In [10]:
URL = "https://opensky-network.org/api/states/all"
load_dotenv()
CLIENT_ID = os.getenv("clientId")
CLIENT_SECRET = os.getenv("clientSecret")
TOKEN_URL = "https://auth.opensky-network.org/auth/realms/opensky-network/protocol/openid-connect/token"

In [11]:
"""Asia region restricted"""
params = {
    "lamin": -10,
    "lamax": 55,
    "lomin": 25,
    "lomax": 180
}

In [12]:
"""Authentication"""
def get_access_token():
    r = requests.post(
        TOKEN_URL,
        headers = {"Content-Type": "application/x-www-form-urlencoded"},
        data = {
            "grant_type": "client_credentials",
            "client_id": CLIENT_ID,
            "client_secret": CLIENT_SECRET
        },
        timeout=10
    )
    r.raise_for_status()
    return r.json()["access_token"]

In [13]:
token = get_access_token()
token_time = time.time()

In [14]:
file_path = "../data/raw/opensky_asia.csv"
os.makedirs(os.path.dirname(file_path), exist_ok=True)
file_exists = os.path.isfile(file_path)

In [15]:
with open("../data/raw/opensky_asia.csv", "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)

    writer.writerow([
        "timestamp",
        "icao24",
        "callsign",
        "latitude",
        "longitude",
        "baro_altitude",
        "velocity",
        "vertical_rate",
        "true_track",
        "on_ground"
    ])

    while True:
        # refresh token every 25 minutes
        if time.time() - token_time > 25 * 60:
            token = get_access_token()
            token_time = time.time()
            print("Token refreshed")

        headers = {
            "Authorization": f"Bearer {token}"
        }

        response = requests.get(
            URL,
            params=params,
            headers=headers,
            timeout=15
        )

        if response.status_code == 429:
            print("Rate limited (429). Backing off for 2 minutes.")
            time.sleep(120)
            continue

        if response.status_code != 200:
            print("Error:", response.status_code)
            time.sleep(30)
            continue

        data = response.json()

        if "time" not in data or "states" not in data:
            print("Unexpected response")
            time.sleep(30)
            continue

        timestamp = datetime.utcfromtimestamp(data["time"])

        count = 0
        for s in data["states"]:
            lat = s[6]
            lon = s[5]

            if lat is None or lon is None:
                continue

            writer.writerow([
                timestamp,
                s[0],      # icao24
                s[1],      # callsign
                lat,
                lon,
                s[7],      # baro_altitude
                s[9],      # velocity
                s[11],     # vertical_rate
                s[10],     # true_track
                s[8],      # on_ground
            ])
            count += 1

        print(f"{timestamp} | aircraft saved: {count}")
        time.sleep(10)

2026-02-13 16:28:11 | aircraft saved: 1136
2026-02-13 16:28:33 | aircraft saved: 1147
2026-02-13 16:28:44 | aircraft saved: 1141
2026-02-13 16:28:54 | aircraft saved: 1143
2026-02-13 16:29:13 | aircraft saved: 1139
2026-02-13 16:29:28 | aircraft saved: 1134
2026-02-13 16:29:36 | aircraft saved: 1136
2026-02-13 16:29:46 | aircraft saved: 1134
2026-02-13 16:29:44 | aircraft saved: 1134
2026-02-13 16:30:08 | aircraft saved: 1139
2026-02-13 16:30:23 | aircraft saved: 1133
2026-02-13 16:30:32 | aircraft saved: 1132
2026-02-13 16:30:42 | aircraft saved: 1131
2026-02-13 16:30:55 | aircraft saved: 1137
2026-02-13 16:31:02 | aircraft saved: 1136
2026-02-13 16:31:12 | aircraft saved: 1130
2026-02-13 16:31:24 | aircraft saved: 1121
2026-02-13 16:31:34 | aircraft saved: 1124
2026-02-13 16:31:45 | aircraft saved: 1124
2026-02-13 16:31:45 | aircraft saved: 1124
2026-02-13 16:32:12 | aircraft saved: 1127
2026-02-13 16:32:26 | aircraft saved: 1126
2026-02-13 16:32:37 | aircraft saved: 1124
2026-02-13 

KeyboardInterrupt: 